In [ ]:
import warnings

from evo.notebooks import ServiceManagerWidget

# Suppress warnings
warnings.filterwarnings("ignore", message=".*SSL.*")
warnings.filterwarnings("ignore", message=".*unclosed.*")
warnings.filterwarnings("ignore", message=".*AuthorizationCodeAuthorizer.*")

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL

# HINT: If the Workspaces list is empty, try running the cell again.

manager = await ServiceManagerWidget.with_auth_code(
    redirect_url=redirect_url,
    client_id=client_id,
).login()

In [ ]:
from helpers.widgets import create_duf_widget

ui = await create_duf_widget(manager)
display(ui)